In [1]:
sparql_endpoint = "http://localhost:9999/blazegraph/namespace/test/sparql"

In [2]:
import os
import pandas as pd
from SPARQLWrapper import SPARQLWrapper, JSON
from tqdm import tqdm

In [3]:
def get_sparql_data(sparql_query):
    print("SPARQL ENDPOINT로부터 데이터를 불러옵니다.")
    endpoint = SPARQLWrapper(sparql_endpoint)
    endpoint.setQuery(sparql_query)
    endpoint.setReturnFormat(JSON)
    result = endpoint.query().convert()
    return pd.json_normalize(result["results"]["bindings"])

In [6]:
def kistib_df():
    df = get_sparql_data(
        """
        PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
        PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
        PREFIX koad: <http://data.kisti.re.kr/administration/administrative-division/def/>
        PREFIX dct: <http://purl.org/dc/terms/>

        SELECT DISTINCT ?division ?name ?code ?romanName ?desc ?createdDate ?codeStat ?area ?population ?numberOfHousehold ?populationOfMale ?populationOfFemale ?genderRatio
        WHERE {
        ?division rdfs:label ?name ;
                    koad:divisionCode ?code ;
                    rdf:type ?type .
        OPTIONAL {?division dct:description ?desc .}
        OPTIONAL {?division dct:created ?createdDate .}
        OPTIONAL {?division koad:divisionCodeStat ?codeStat .}
        OPTIONAL {?division koad:romanizedLabel ?romanName .}
        OPTIONAL {?division koad:population ?population .}
        OPTIONAL {?division koad:area ?area .}
        OPTIONAL {?division koad:numberOfHousehold ?numberOfHousehold .}
        OPTIONAL {?division koad:populationOfMale ?populationOfMale ; .}
        OPTIONAL {?division koad:populationOfFemale ?populationOfFemale .}
        OPTIONAL {?division koad:genderRatio ?genderRatio .}
        FILTER (regex(str(?division), 'administrative-division'))
        } 
        ORDER BY ?code
        # LIMIT 10
        """
    )

    # value 컬럼만 넣기
    columns = [n for n in df.columns if n.find(".value") != -1]
    df = df[columns]

    # 컬럼명에서 .value 삭제
    df = df.rename(columns=lambda col: col.replace(".value", ""))
    df = df.fillna("None")
    print(df.head())
    print(df.columns)
    df.to_csv("final-data/csv/koad.csv", encoding="utf-8", index=False)
    return df

In [7]:
kistib_df()

SPARQL ENDPOINT로부터 데이터를 불러옵니다.
                                            division   name        code  \
0  http://data.kisti.re.kr/administration/adminis...  서울특별시  1100000000   
1  http://data.kisti.re.kr/administration/adminis...    종로구  1111000000   
2  http://data.kisti.re.kr/administration/adminis...    청운동  1111010100   
3  http://data.kisti.re.kr/administration/adminis...    신교동  1111010200   
4  http://data.kisti.re.kr/administration/adminis...    궁정동  1111010300   

   romanName                                               desc createdDate  \
0      Seoul  서울은 한국의 수도인 동시에 동북아 비즈니스의 중심지이며 한국 산업의 집적지로 아시...  1988-04-23   
1  Jongno-gu  종로구는 서울특별시의 중북부에 있으며 동쪽으로 동대문구와 성북구, 서쪽에 서대문구,...  1988-04-23   
2       None                                               None        None   
3       None                                               None        None   
4       None                                               None        None   

  codeStat    area population numberOfHouse

,division,name,code,romanName,desc,createdDate,codeStat,area,population,numberOfHousehold,populationOfMale,populationOfFemale,genderRatio
0,http://data.kisti.re.kr/administration/adminis...,서울특별시,1100000000,Seoul,서울은 한국의 수도인 동시에 동북아 비즈니스의 중심지이며 한국 산업의 집적지로 아시...,1988-04-23,11,605.24,9494807,4464840,4605655,4889152,0.94
1,http://data.kisti.re.kr/administration/adminis...,종로구,1111000000,Jongno-gu,"종로구는 서울특별시의 중북부에 있으며 동쪽으로 동대문구와 성북구, 서쪽에 서대문구,...",1988-04-23,11010,23.91,143624,73728,69513,74111,0.94
2,http://data.kisti.re.kr/administration/adminis...,청운동,1111010100,None,None,None,None,None,None,None,None,None,None
3,http://data.kisti.re.kr/administration/adminis...,신교동,1111010200,None,None,None,None,None,None,None,None,None,None
4,http://data.kisti.re.kr/administration/adminis...,궁정동,1111010300,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
22721,http://data.kisti.re.kr/administration/adminis...,서홍동,5013058000,Seohong-dong,None,2006-07-01,39020580,None,11254,4644,5556,5698,0.98
22722,http://data.kisti.re.kr/administration/adminis...,대륜동,5013059000,Daeryun-dong,None,2006-07-01,39020590,None,15411,6923,7802,7609,1.03
22723,http://data.kisti.re.kr/administration/adminis...,대천동,5013060000,Daecheon-dong,None,2006-07-01,39020600,None,13802,6460,6980,6822,1.02
22724,http://data.kisti.re.kr/administration/adminis...,중문동,5013061000,Jungmun-dong,None,2006-07-01,39020610,None,12217,5903,6197,6020,1.03
